In [8]:
import pandas as pd
from io import StringIO
from typing import Optional, List

class DataLoader:
    """
    Parses European wholesale electricity price data, allowing filtering
    by country and date range.
    """
    def __init__(self, file_path="./data/european_wholesale_electricity_price_data_daily.csv"):
        """
        Initializes the parser and loads the data.

        Args:
            file_path (str): The path to the CSV file.
        """
        self.file_path = file_path
        self.data = self._load_data()

    def _load_data(self):
        """Loads and preprocesses the data from the CSV file."""
        try:
            df = pd.read_csv(self.file_path)
            # Convert 'Date' column to datetime objects
            df['Date'] = pd.to_datetime(df['Date'])
            # Drop ISO3 Code column
            df.drop(columns={'ISO3 Code'}, inplace=True)
            # Rename price column for easier access
            df.rename(columns={'Price (EUR/MWhe)': 'Price'}, inplace=True)
            print(f"Data loaded successfully from {self.file_path}")
            return df
        except FileNotFoundError:
            print(f"Error: File not found at {self.file_path}")
            return None
        except KeyError as e:
            print(f"Error: Expected column '{e}' not found in the CSV.")
            return None
        except Exception as e:
            print(f"Error loading or processing file: {e}")
            return None

    def get_data_by_country_and_range(self, time_range:str, country=None):
        """
        Filters the data for a specific country and time range.

        Args:
            country (str): The name of the country to filter by (e.g., 'Germany').
            time_range (str): A string representing the date range in the format
                              'YYYY-MM-DD,YYYY-MM-DD'.

        Returns:
            pandas.DataFrame: A DataFrame containing the filtered data,
                              or None if an error occurs or no data is found.
        """
        if self.data is None:
            print("Error: Data not loaded.")
            return None

        try:
            start_date_str, end_date_str = time_range.split(',')
            start_date = pd.to_datetime(start_date_str.strip())
            end_date = pd.to_datetime(end_date_str.strip())
        except ValueError:
            print("Error: Invalid time_range format. Please use 'YYYY-MM-DD,YYYY-MM-DD'.")
            return None
        except Exception as e:
             print(f"Error parsing time range: {e}")
             return None

        outputData = self.data.copy()
        # If a country is specified, filter the data by country, if not use all data
        if country is not None:
            outputData = self.data[self.data['Country'].str.lower() == country.lower()]

        # Filter by date range (inclusive)
        filtered_data = outputData[
            (outputData['Date'] >= start_date) & (outputData['Date'] <= end_date)
        ]

        if filtered_data.empty:
            print(f"Warning: No data found for country '{country}' within the range {time_range}.")
            return pd.DataFrame() # Return empty DataFrame

        return filtered_data.copy() # Return a copy to avoid SettingWithCopyWarning

    def get_all_data(self):
        """
        Returns the entire dataset.

        Returns:
            pandas.DataFrame: The entire dataset.
        """
        if self.data is None:
            print("Error: Data not loaded.")
            return None
        return self.data.copy()

    def get_country_list(self):
        """
        Returns a list of unique countries in the dataset.

        Returns:
            list: A list of unique country names.
        """
        if self.data is None:
            print("Error: Data not loaded.")
            return None
        return self.data['Country'].unique().tolist()
    
    def get_price_matrix(
        self,
        time_range: str,
        countries: List[str],
        fill_method: Optional[str] = None
    ) -> pd.DataFrame:
        """
        Returns a price matrix where:
        - Rows = dates
        - Columns = countries
        - Values = daily electricity prices

        Parameters:
        - time_range (str): e.g. "2021-05-10,2021-05-16"
        - countries (List[str]): list of country names to include
        - fill_method (Optional[str]): 'ffill', 'bfill', or None

        Returns:
        - pd.DataFrame: index=date, columns=country names, values=prices
        """
        start_date, end_date = time_range.split(",")

        # Filter the master data once
        df = self.data.copy()
        df = df[df["Country"].isin(countries)]
        df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]

        # Pivot: index=date, columns=country, values=price
        price_matrix = df.pivot(index="Date", columns="Country", values="Price").sort_index()

        # Handle missing data
        if fill_method:
            price_matrix = price_matrix.fillna(method=fill_method)
        else:
            price_matrix = price_matrix.dropna()

        return price_matrix

In [9]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import yfinance as yf
import matplotlib.pyplot as plt

class CointegrationResidualGenerator:
    def __init__(self, price_data: pd.DataFrame, risk_free_rate_annual: float = 0.01):
        self.price_data = price_data
        self.risk_free_rate_daily = risk_free_rate_annual / 252
        self.returns = self._compute_excess_returns(price_data)
        self.cumulative_returns = self._compute_cumulative_returns(self.returns)
        self.asset_residuals = pd.DataFrame(index=self.cumulative_returns.index)
        self.betas = {}  # Store betas for each asset

    def _compute_excess_returns(self, price_data: pd.DataFrame) -> pd.DataFrame:
        """Computes daily excess log returns."""
        log_returns = np.log(price_data / price_data.shift(1)).dropna()
        excess_returns = log_returns.subtract(self.risk_free_rate_daily)
        return excess_returns

    def _compute_cumulative_returns(self, returns: pd.DataFrame) -> pd.DataFrame:
        """Computes cumulative returns."""
        cumulative_returns = returns.cumsum()
        return cumulative_returns

    def compute_all_asset_residuals(self):
        """Computes residuals for each asset treated as dependent variable."""
        for target_asset in self.cumulative_returns.columns:
            # Set current asset as dependent variable
            y = self.cumulative_returns[target_asset].values.reshape(-1, 1)
            X = self.cumulative_returns.drop(columns=[target_asset]).values
            X_cols = self.cumulative_returns.drop(columns=[target_asset]).columns

            # Fit linear regression: y ~ X
            model = LinearRegression().fit(X, y)
            betas = model.coef_[0]
            intercept = model.intercept_[0]

            # Predict values and compute residuals for this asset
            y_pred = model.predict(X).flatten()
            residuals = y.flatten() - y_pred

            # Store residuals in DataFrame
            self.asset_residuals[target_asset] = residuals
            # Store betas for this asset
            beta_series = pd.Series(betas, index=X_cols)
            beta_series['Intercept'] = intercept
            self.betas[target_asset] = beta_series

    def get_asset_residuals(self) -> pd.DataFrame:
        """Returns residuals for all assets."""
        if self.asset_residuals.empty:
            raise ValueError("Asset residuals not yet computed.")
        return self.asset_residuals

    def get_betas_for_asset(self, asset: str) -> pd.Series:
        """Returns betas used to form residuals for a specific asset."""
        if asset not in self.betas:
            raise ValueError(f"Betas for asset '{asset}' not found. Compute residuals first.")
        return self.betas[asset]

    def prepare_cnn_input_from_residuals(self, window: int = 30):
        """
        Prepares CNN input data by creating rolling cumulative residuals.
        
        Returns:
        - 3D numpy array: [samples, window, assets]
        """
        if self.asset_residuals.empty:
            raise ValueError("Asset residuals not yet computed.")

        cnn_input_list = []

        for start_idx in range(len(self.asset_residuals) - window + 1):
            # Slice window of residuals
            window_residuals = self.asset_residuals.iloc[start_idx:start_idx + window]
            # Cumulative sum within the window
            cumulative_window = window_residuals.cumsum()
            # Store the result
            cnn_input_list.append(cumulative_window.values)

        # Convert to 3D numpy array: [samples, window, assets]
        cnn_input_array = np.array(cnn_input_list)
        return cnn_input_array


In [10]:
import torch
import torch.nn as nn


class CNN(nn.Module):
    def __init__(self, input_length, num_features, num_filters= 8, num_classes=2, filter_size=2):
        """
        Initialize the CNN model based on the equations in the paper.
        
        Args:
            input_length (int): Length of the input sequence (L in the equations)
            num_features (int): Number of features per time step
            num_classes (int): Number of output classes
        """
        super(CNN, self).__init__()
        
        self.num_filters = num_filters  # Number of filters (D = 8 according to equation)
        self.filter_size = filter_size  # Filter size (filter_size = 2 according to equation)
        
        # First convolutional layer (Equation 3)
        # Input shape: [batch_size, num_features, input_length]
        self.conv1 = nn.Conv1d(
            in_channels=num_features,
            out_channels=self.num_filters,
            kernel_size=self.filter_size,
            stride=1,
            padding=0
        )
        
        # Second convolutional layer (Equation 4)
        self.conv2 = nn.Conv1d(
            in_channels=self.num_filters,
            out_channels=self.num_filters,
            kernel_size=self.filter_size,
            stride=1,
            padding=0
        )
        
        # Calculate output sizes after convolutions
        L_after_conv1 = input_length - self.filter_size + 1
        L_after_conv2 = L_after_conv1 - self.filter_size + 1
        
        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(self.num_filters * L_after_conv2, num_classes)
        
        # Activation
        self.relu = nn.ReLU()
        
    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, num_features, input_length]
                             Represents the x_i(t) vector in Equation 2
        
        Returns:
            torch.Tensor: Output predictions
        """
        # Store original input for skip connection (Equation 5)
        x_original = x
        
        # First convolutional layer (Equation 3)
        x = self.relu(self.conv1(x))
        
        # Second convolutional layer (Equation 4)
        x = self.relu(self.conv2(x))
        
        # Skip connection (Equation 5)
        # Need to adjust dimensions for skip connection
        # Cut or pad x_original to match x dimensions
        if x_original.shape[2] > x.shape[2]:
            # If original is longer, cut it
            diff = x_original.shape[2] - x.shape[2]
            x_skip = x_original[:, :, diff:]
        else:
            # If original is shorter, this would require padding
            # For simplicity, we'll just use the original shape
            x_skip = x_original
        
        # Apply the skip connection if dimensions match
        if x.shape == x_skip.shape:
            x = x + x_skip
        
        # Flatten and pass through fully connected layer
        x = self.flatten(x)
        x = self.fc(x)
        
        return x
    
    def get_parameters(self):
        return self.parameters()


In [67]:
import torch
import torch.nn as nn # building blocks for neural networks
import torch.nn.functional as F # access to functions like ReLU, sigmoid, etc.

class FNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=32, output_dim=31):
        """
        Initialize the Feedforward Neural Network.
        
        Args:
            input_dim (int): Input dimension (matches CNN output)
            hidden_dim (int): Hidden layer dimension
            output_dim (int): Number of output weights (one per country)
        """
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, input_dim]
        
        Returns:
            torch.Tensor: Output tensor of shape [batch_size, output_dim]
        """
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        w_eps = self.out(x)  # Shape: [batch_size, output_dim]
        return w_eps
    
# equation (11) from project framework
def soft_normalize(weights):
    """
    Normalize allocation weights using L1 norm (sum of absolute values).
    weights: Tensor of shape [batch_size, 1]
    Returns: Normalized weights of shape [batch_size, 1]
    """
    l1_norm = torch.sum(torch.abs(weights), dim=0, keepdim=True) + 1e-8 # avoid division by zero
    normalized_weights = weights / l1_norm
    return normalized_weights

In [68]:
import numpy as np

class BacktestSharpeEvaluator:
    def __init__(self):
        self.portfolio_returns = []

    def add_return(self, r: float):
        """Add a single next-day portfolio return."""
        self.portfolio_returns.append(r)

    def add_returns(self, returns: list):
        """Add a list of next-day portfolio returns."""
        self.portfolio_returns.extend(returns)

    def reset(self):
        """Reset the stored returns."""
        self.portfolio_returns = []

    def calculate_sharpe(self, returns=None, risk_free_rate=0.0):
        """
        Calculate Sharpe Ratio from stored or passed-in returns.
        Sharpe Ratio = (mean - risk-free) / std deviation
        """
        r = self.portfolio_returns if returns is None else returns
        r = np.array(r)
        if len(r) == 0 or np.std(r) == 0:
            return np.nan
        excess_returns = r - risk_free_rate
        return np.mean(excess_returns) / np.std(excess_returns)

    def normalize_weights_l1(self, raw_weights, phi=None):
        """
        Normalize raw weights using Ordoñez's method:
        w_normalized = (w_raw^T * phi) / ||w_raw^T * phi||_1

        Parameters:
            raw_weights: numpy array of shape (n_assets,)
            phi: optional transformation matrix (e.g., identity or mapping from factor to asset space)

        Returns:
            L1-normalized weights: numpy array of shape (n_assets,)
        """
        if phi is None:
            phi = np.eye(len(raw_weights))  # default to identity if no mapping provided
        raw = raw_weights.T @ phi
        norm = np.sum(np.abs(raw))
        if norm == 0:
            return np.zeros_like(raw)
        return raw / norm

    def compute_portfolio_return(self, raw_weights, next_day_returns, phi=None):
        """
        Normalize weights, compute and store the next-day portfolio return.

        Parameters:
            raw_weights: numpy array of shape (n_assets,)
            next_day_returns: numpy array of shape (n_assets,)
            phi: optional transformation matrix

        Returns:
            Computed return (float)
        """
        w = self.normalize_weights_l1(raw_weights, phi)
        r = float(np.dot(w, next_day_returns))
        self.add_return(r)
        return r

In [69]:
def sharpe_ratio_loss(returns, risk_free_rate=0.0):
    """
    Custom loss function to maximize the Sharpe Ratio.
    Args:
        returns (torch.Tensor): Predicted returns
        risk_free_rate (float): Risk-free rate for Sharpe calculation
    Returns:
        torch.Tensor: Negative Sharpe Ratio (to minimize)
    """
    excess_returns = returns - risk_free_rate
    mean_excess = torch.mean(excess_returns)
    std_excess = torch.std(excess_returns, unbiased=False) + 1e-6  # epsilon for stability
    sharpe_ratio = mean_excess / std_excess
    return -sharpe_ratio

In [ ]:
import torch.optim as optim

class PortfolioOptimizer:
    """
    A class to train and evaluate a CNN+FNN model for portfolio optimization,
    generating weights to maximize the Sharpe ratio based on electricity price residuals.
    """
    def __init__(self, cnn_input_array: np.array, next_day_returns: pd.DataFrame, device=None, num_filters=8, filter_size=2, hidden_dim=32,
                 lr=0.001, num_epochs=1000, batch_size=32):
        """
        Initialize the portfolio optimizer with input data and hyperparameters.

        Args:
            cnn_input_array (np.ndarray): Input residuals with shape [samples, num_countries, window_size].
            next_day_returns (pd.DataFrame): Next-day returns with shape [samples, num_countries].
            device (torch.device, optional): Device for computation (cuda, mps, or cpu). Defaults to auto-detection.
            num_filters (int): Number of CNN filters.
            filter_size (int): Size of CNN convolutional filters.
            hidden_dim (int): Hidden dimension for CNN output and FNN layers.
            lr (float): Learning rate for the optimizer.
            num_epochs (int): Number of training epochs.
            batch_size (int): Batch size for training.
        """
        # Set device (GPU, MPS, or CPU)
        if device is None:
            if torch.cuda.is_available():
                device = torch.device("cuda")
            elif torch.backends.mps.is_available():
                device = torch.device("mps")
            else:
                device = torch.device("cpu")
        self.device = device

        # Data dimensions
        self.num_countries = cnn_input_array.shape[1]  # Number of countries (31)
        self.window_size = cnn_input_array.shape[2]    # Window size for residuals (30)
        self.num_samples = cnn_input_array.shape[0]    # Number of samples (329, or the number of days)

        # Hyperparameters
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.hidden_dim = hidden_dim
        self.lr = lr
        self.num_epochs = num_epochs
        self.batch_size = batch_size

        # Convert input data to PyTorch tensors
        self.cnn_input_tensor = torch.FloatTensor(cnn_input_array).to(self.device)
        self.next_day_returns_tensor = torch.FloatTensor(next_day_returns.values).to(self.device)

        # Initialize models
        self.cnn_model = self._initialize_cnn()
        self.fnn_model = self._initialize_fnn()

        # Initialize optimizer
        self.optimizer = optim.Adam(
            list(self.cnn_model.parameters()) + list(self.fnn_model.parameters()),
            lr=self.lr
        )

        # Initialize evaluator for Sharpe ratio calculation
        self.evaluator = BacktestSharpeEvaluator()

    def _initialize_cnn(self):
        """
        Initialize the CNN model.

        Returns:
            CNN: Initialized CNN model on the specified device.
        """
        # Calculate output sizes after convolutions
        L_after_conv1 = self.window_size - self.filter_size + 1  # e.g., 30 - 2 + 1 = 29
        L_after_conv2 = L_after_conv1 - self.filter_size + 1    # e.g., 29 - 2 + 1 = 28

        cnn = CNN(
            input_length=self.window_size,
            num_features=self.num_countries,
            num_filters=self.num_filters,
            num_classes=self.hidden_dim,  # Output matches FNN input dimension
            filter_size=self.filter_size
        ).to(self.device)
        return cnn

    def _initialize_fnn(self):
        """
        Initialize the FNN model.

        Returns:
            FNN: Initialized FNN model on the specified device.
        """
        fnn = FNN(
            input_dim=self.hidden_dim,
            hidden_dim=self.hidden_dim,
            output_dim=self.num_countries  # One weight per country
        ).to(self.device)
        return fnn

    def soft_normalize(self, weights):
        """
        Normalize weights using L1 norm (sum of absolute values = 1).

        Args:
            weights (torch.Tensor): Raw weights with shape [batch_size, num_countries].

        Returns:
            torch.Tensor: Normalized weights with shape [batch_size, num_countries].
        """
        l1_norm = torch.sum(torch.abs(weights), dim=1, keepdim=True) + 1e-8  # Avoid division by zero
        normalized_weights = weights / l1_norm
        return normalized_weights

    def sharpe_ratio_loss(self, returns, risk_free_rate=0.0):
        """
        Compute negative Sharpe ratio as loss for optimization.

        Args:
            returns (torch.Tensor): Portfolio returns with shape [batch_size].
            risk_free_rate (float): Risk-free rate (default: 0.0).

        Returns:
            torch.Tensor: Negative Sharpe ratio (scalar).
        """
        excess_returns = returns - risk_free_rate
        mean_excess = torch.mean(excess_returns)
        std_excess = torch.std(excess_returns, unbiased=False) + 1e-5  # Avoid division by zero
        sharpe_ratio = mean_excess / std_excess
        return -sharpe_ratio

    def train(self):
        """
        Train the CNN+FNN model to maximize the Sharpe ratio.

        Returns:
            list: Portfolio returns from the final evaluation.
        """
        for epoch in range(self.num_epochs):
            # Set models to training mode
            self.cnn_model.train()
            self.fnn_model.train()

            # Process data in batches
            for batch_idx in range(0, len(self.cnn_input_tensor), self.batch_size):
                # Extract batch
                batch_end = min(batch_idx + self.batch_size, len(self.cnn_input_tensor))
                batch_inputs = self.cnn_input_tensor[batch_idx:batch_end]  # Shape: [batch_size, num_countries, window_size]
                batch_returns = self.next_day_returns_tensor[batch_idx:batch_end]  # Shape: [batch_size, num_countries]

                # Zero gradients
                self.optimizer.zero_grad()

                # Forward pass: CNN processes residuals
                cnn_output = self.cnn_model(batch_inputs)  # Shape: [batch_size, hidden_dim]

                # Forward pass: FNN generates weights
                weights = self.fnn_model(cnn_output)  # Shape: [batch_size, num_countries]

                # Normalize weights to sum to 1 (L1 norm)
                normalized_weights = self.soft_normalize(weights)  # Shape: [batch_size, num_countries]

                # Compute portfolio returns (dot product of weights and returns)
                portfolio_returns = torch.sum(normalized_weights * batch_returns, dim=1)  # Shape: [batch_size]

                # Compute loss (negative Sharpe ratio)
                loss = self.sharpe_ratio_loss(portfolio_returns)

                # Backward pass and optimization
                loss.backward()
                self.optimizer.step()

            
            if epoch % 10 == 0:
                # Evaluate Sharpe ratio for every 10th epoch
                sharpe = self._evaluate_epoch()
                print(f"Epoch {epoch}, Sharpe Ratio: {sharpe:.4f}")

        # Perform final evaluation
        final_sharpe, portfolio_returns = self.evaluate_final()
        print(f"\nFinal Sharpe Ratio: {final_sharpe:.4f}")

        return portfolio_returns

    def _evaluate_epoch(self):
        """
        Evaluate the model for one epoch, computing the Sharpe ratio.

        Returns:
            float: Sharpe ratio for the epoch.
        """
        self.evaluator.reset()
        self.cnn_model.eval()
        self.fnn_model.eval()

        with torch.no_grad():
            for i in range(0, len(self.cnn_input_tensor), self.batch_size):
                batch_end = min(i + self.batch_size, len(self.cnn_input_tensor))
                batch_inputs = self.cnn_input_tensor[i:batch_end]
                batch_returns = self.next_day_returns_tensor[i:batch_end]

                # Forward pass
                cnn_output = self.cnn_model(batch_inputs)
                weights = self.fnn_model(cnn_output)
                normalized_weights = self.soft_normalize(weights)

                # Compute portfolio returns
                portfolio_returns = torch.sum(normalized_weights * batch_returns, dim=1)
                portfolio_returns_np = portfolio_returns.cpu().numpy()

                # Store returns in evaluator
                for return_val in portfolio_returns_np:
                    self.evaluator.add_return(return_val)

        return self.evaluator.calculate_sharpe()

    def evaluate_final(self):
        """
        Perform final evaluation on all samples.

        Returns:
            tuple: (final Sharpe ratio, list of portfolio returns).
        """
        self.evaluator.reset()
        self.cnn_model.eval()
        self.fnn_model.eval()

        with torch.no_grad():
            for i in range(len(self.cnn_input_tensor)):
                inputs = self.cnn_input_tensor[i:i+1]  # Shape: [1, num_countries, window_size]
                returns = self.next_day_returns_tensor[i:i+1]  # Shape: [1, num_countries]

                # Forward pass
                cnn_output = self.cnn_model(inputs)
                weights = self.fnn_model(cnn_output)
                normalized_weights = self.soft_normalize(weights)

                # Compute portfolio return
                portfolio_return = torch.sum(normalized_weights * returns, dim=1).item()
                self.evaluator.add_return(portfolio_return)

        final_sharpe = self.evaluator.calculate_sharpe()
        return final_sharpe, self.evaluator.portfolio_returns

In [93]:

# MAIN FUNCTION
### -------------- PARSING DATA -------------- ###
parser = DataLoader()

# Get list of countries
print("\n--- List of Countries ---")
all_countries_list = parser.get_country_list()
print(all_countries_list)

# Get daily price matrix for all countries for the entire year 2021
price_matrix = parser.get_price_matrix(
    time_range="2021-01-01,2021-12-31",
    countries=all_countries_list,
    fill_method="ffill"
)

# Get the raw daily returns for the price matrix
returns = price_matrix.pct_change().dropna()



### -------------- COINTEGRATION RESIDUALS -------------- ###
# Create an instance of the CointegrationResidualGenerator
residual_generator = CointegrationResidualGenerator(price_matrix)

residual_generator.compute_all_asset_residuals()

# Get residuals
asset_residuals = residual_generator.get_asset_residuals()

# Get the input for CNN
# cnn_input contains a set of 329 data samples, each sample represents 30-day cumulative residuals for the 31 countries
cumulative_residual_window = 30
cnn_input = residual_generator.prepare_cnn_input_from_residuals(window=cumulative_residual_window)

# Get the start index of the first 30-day cumulative residuals in the returns DataFrame
start_idx_in_returns = returns.index.get_loc(asset_residuals.index[0])
num_samples = len(asset_residuals) - cumulative_residual_window + 1
next_day_indices = [start_idx_in_returns + i + cumulative_residual_window for i in range(num_samples)]

# Get the next-day returns for the corresponding indices
# The next-day returns are the returns for the day after the last day of each 30-day window
# For example, if the first 30-day window ends on index 0, the next day return is at index 1
# If the second 30-day window ends on index 1, the next day return is at index 2, and so on.
next_day_returns = returns.iloc[next_day_indices]




### -------------- FEED THE 30-DAY CUMULATIVE RESIDUALS OF EVERY COUNTRY TO CNN+FNN -------------- ###
# Transform cnn_input to be compatible with the CNN input shape
cnn_input_array = cnn_input.transpose(0, 2, 1) # [samples, features, window]

# FILIP'S SECTION: CNN+FNN
# Hey Filip, this is the section where you can add your code to train the CNN+FNN model.
# cnn_input_array essentially contains 329 training data points, each data point is 30-day cumulative residuals for the 31 countries.
# So, for one "data point" you would feed the set of 30-day cumulative residuals for every country to the CNN+FNN model.
# One data point should result in a set of weights for each of the 31 countries.
# Each set of weights is used to calculate one next-day portfolio return.
# Repeat this for all 329 data points to get a set of 329 portfolio returns.
# You can then use these portfolio returns to calculate the Sharpe ratio
# Optimize the CNN+FFN to maximize the Sharpe ratio.

# Set the device to GPU if available, otherwise MPS (for Mac silicon) or CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

torch.manual_seed(1)  # For reproducibility

# Create the PortfolioOptimizer instance and train the model
# NOTE: We need to have some validation data to evaluate the model performance
# otherwise we will be overfitting the model to the training data
optimizer = PortfolioOptimizer(cnn_input_array, next_day_returns, batch_size=1000, num_epochs=100)
portfolio_returns = optimizer.train()

### -------------- GET THE WEIGHTS OUTPUTTED FROM CNN+FNN -------------- ###
# Initializing the Sharpe ratio evaluator
#evaluator = BacktestSharpeEvaluator()

# Get the weight outputted from the CNN+FNN model
# One weight for each country
#weights = np.array([]) # CHANGE THIS TO THE ACTUAL WEIGHTS OUTPUTTED FROM THE CNN+FNN MODEL

# Normalize the weights using L1 normalization
# This is done to ensure that the portfolio is dollar-neutral
#normalized_weights = evaluator.normalize_weights_l1(weights)

# Multiply the normalized weights (vector) with the next-day returns (vector) to get the portfolio return
#next_day_portfolio_return = evaluator.compute_portfolio_return(normalized_weights, next_day_returns.iloc[0].values)

# Store the next day portfolio return 
#evaluator.add_return(next_day_portfolio_return)

# Repeat the above step for all 329 data points, adding the portfolio returns to the evaluator for each data point

# Once all portfolio returns are calculated, you can calculate the Sharpe ratio

# Train the model to optimize the Sharpe Ratio


Data loaded successfully from ./data/european_wholesale_electricity_price_data_daily.csv

--- List of Countries ---
['Austria', 'Belgium', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'Bulgaria', 'Serbia', 'Croatia', 'Montenegro', 'North Macedonia', 'Ireland']
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Epoch 0, Sharpe Ratio: 0.0005
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])


/var/folders/_l/8ljm4_2n0cl02xgf0_pbjrtw0000gn/T/ipykernel_25650/797593302.py:142: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_matrix = price_matrix.fillna(method=fill_method)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Epoch 10, Sharpe Ratio: 0.0348
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Epoch 20, Sharpe Ratio: 0.0520
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized weights: torch.Size([329, 31])
Normalized wei